In [7]:
pip install mne h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 15.6 MB/s  0:00:00 eta 0:00:01

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import scipy.io as sio
import numpy as np

# Load MATLAB NIRS file
mat_data = sio.loadmat('S133_run03.nirs')

# Extract key data
data = mat_data['d']  # Time-series data (16875 x 48 channels)
time = mat_data['t'].flatten()  # Time vector
ml = mat_data['ml']  # Measurement list
sd = mat_data['SD'][0, 0]  # Source-detector configuration
info = mat_data['systemInfo'][0, 0]  # System info

print("NIRS Data loaded successfully!")
print(f"Data shape: {data.shape}")
print(f"Time points: {len(time)}")
print(f"Measurement list shape: {ml.shape}")
print(f"Duration: {time[-1] - time[0]:.2f} seconds")
print(f"Sampling rate: {1 / np.mean(np.diff(time)):.1f} Hz")

NIRS Data loaded successfully!
Data shape: (16875, 48)
Time points: 16875
Measurement list shape: (48, 4)
Duration: 674.96 seconds
Sampling rate: 25.0 Hz


In [4]:
import scipy.io as sio

# Load the MATLAB file
data = sio.loadmat('S133_run03.nirs')
print("Keys in MATLAB file:")
for key in data.keys():
    if not key.startswith('__'):
        print(f"  - {key}: {type(data[key]).__name__}")
        if hasattr(data[key], 'shape'):
            print(f"    Shape: {data[key].shape}")

Keys in MATLAB file:
  - SD: ndarray
    Shape: (1, 1)
  - ml: ndarray
    Shape: (48, 4)
  - d: ndarray
    Shape: (16875, 48)
  - t: ndarray
    Shape: (16875, 1)
  - dStd: ndarray
    Shape: (0, 0)
  - aux: ndarray
    Shape: (16875, 8)
  - s: ndarray
    Shape: (16875, 1)
  - tdml: ndarray
    Shape: (48, 16875)
  - systemInfo: ndarray
    Shape: (1, 1)
